In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# ✅ 1. Lade das Modell (automatisch heruntergeladen, wenn es noch nicht lokal ist)
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

c:\Users\haabs\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ✅ 2. Mobilitätswörterpool definieren (aus Datei einlesen)
def load_mobility_keywords(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        keywords = [line.strip() for line in file if line.strip()]  # Leere Zeilen ignorieren
    return keywords

# Lade die Mobilitätsbegriffe aus der Datei
mobility_keywords_file = 'data/mobility_keywords_pool.txt'
mobility_keywords = load_mobility_keywords(mobility_keywords_file)

In [4]:
# ✅ 3. Keywords in Vektoren umwandeln
keyword_embeddings = model.encode(mobility_keywords)

In [5]:
# ✅ 4. Funktion zur Ähnlichkeitsprüfung
def check_similarity(text, keyword_embeddings, model, threshold=0.5):
    text_embedding = model.encode([text])
    similarities = cosine_similarity(text_embedding, keyword_embeddings)
    max_similarity = np.max(similarities)
    return max_similarity >= threshold, max_similarity

In [ ]:
import pandas as pd
import os

# ✅ Funktion zur Analyse des Datasets
def analyze_dataset(file_path, keyword_embeddings, model, threshold=0.5):
    # Prüfen, ob die Datei existiert
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Datei nicht gefunden: {file_path}")
    
    # Lade die Datei
    df = pd.read_csv(file_path)
    
    # Überprüfen, ob Daten korrekt geladen wurden
    if df.empty:
        raise ValueError("Die Datei ist leer oder wurde nicht korrekt geladen.")
    
    # Neue Spalten für Analyse-Ergebnisse
    labels = []
    scores = []

    for description in df.iloc[:, 0]:  # Gehe jede Zeile der ersten Spalte durch
        is_mobility, similarity_score = check_similarity(str(description), keyword_embeddings, model, threshold)
        label = 'Mobilitätsdaten' if is_mobility else 'Nicht Mobilitätsdaten'
        labels.append(label)
        scores.append(similarity_score)

    # Speichere die Ergebnisse in neuen Spalten
    df['Analyse Ergebnis'] = labels
    df['Ähnlichkeit Score'] = scores

    # Speichere die aktualisierte Datei im selben Ordner wie die Eingabedatei
    output_file = os.path.join(os.path.dirname(file_path), 'data/analysed_datasets.csv')
    df.to_csv(output_file, index=False)
    
    print(f"✅ Analyse abgeschlossen. Datei gespeichert unter: {output_file}")
    return output_file

# 📌 Pfade anpassen
input_file_path = 'data/zufaellige_zeilen2.txt'  # <-- Pfad zu deiner Datei hier einfügen

# 📌 Datei analysieren (modifiziere diese Zeile mit deinem Modell & Embeddings)
output_file_path = analyze_dataset(input_file_path, keyword_embeddings, model)


✅ Analyse abgeschlossen. Datei gespeichert unter: analysed_datasets.csv


# wie kann man diese ergebnisse darstellen?